In [1]:
import pandas as pd
import numpy as np
import csv

# União dos atributos de localidade

In [2]:
def preProcessamento(df):
    df = salario.copy() 
    local = []
    cidades = salario.cidade
    estados = salario.estado
    paises = salario.pais

    for i in range(0,len(cidades)):
        local.append(str('{}, {}, {}'.format(cidades[i], estados[i], paises[i])))
        
    pd.DataFrame(local)
    df = df.drop("cidade",axis=1)
    df = df.drop("estado",axis=1)
    df = df.drop("pais",axis=1)

    df['local'] = local

    sequence = ['id', 'local', 'genero', 'data', 'salario']
    df = df.reindex(columns=sequence)
    
    return df

# Árvore de Localização

In [3]:
def gerarArvoreLocal(df):
    nivel0Local = df.local.unique().tolist()
    
    lista_local = []
    nivel1Local = df["local"].str.split(", ", n=2, expand=True)
    estado = nivel1Local.iloc[:,1]
    pais = nivel1Local.iloc[:,2]
    for i in range(0, nivel1Local.shape[0]):
        lista_local.append(["*, "+estado[i]+", "+pais[i]])

    nivel1Local = pd.DataFrame(lista_local)
    nivel1Local = nivel1Local[0].unique().tolist()
    
    nivel2Local = []
    for i in range(0, pais.shape[0]):
        nivel2Local.append("*, *, "+pais[i])

    nivel2Local = pd.DataFrame(nivel2Local)
    nivel2Local = nivel2Local[0].unique().tolist()
    
    nivel3Local = "*, *, *"
    
    arvLocal = [nivel0Local, nivel1Local, nivel2Local, nivel3Local]
    
    return arvLocal

In [4]:
def buscaNivelLocal(local, arvLocal):
    for nivel in range(0, len(arvLocal)):
        if(local in arvLocal[nivel]):
            return nivel

# Árvore da Data

In [5]:
def gerarArvoreData(df):
    nivel0Data = df.data.unique().tolist()
    
    lista_data = []
    nivel1Data = df["data"].str.split("/", n=2, expand=True)
    mes = nivel1Data.iloc[:,1]
    ano = nivel1Data.iloc[:,2]
    for i in range(0, nivel1Data.shape[0]):
        lista_data.append(["**/"+mes[i]+"/"+ano[i]])

    nivel1Data = pd.DataFrame(lista_data)
    nivel1Data = nivel1Data[0].unique().tolist()
    
    nivel2Data = []
    for i in range(0, ano.shape[0]):
        nivel2Data.append("**/**/"+ano[i])

    nivel2Data = pd.DataFrame(nivel2Data)
    nivel2Data = nivel2Data[0].unique().tolist()
    
    nivel3Data = "**/**/****"
    
    arvData = [nivel0Data, nivel1Data, nivel2Data, nivel3Data]
    
    return arvData

In [6]:
def buscaNivelData(data, arvData):
    for nivel in range(0, len(arvData)):
        if(data in arvData[nivel]):
            return nivel

# Árvore do Gênero

In [7]:
def gerarArvoreGenero(df):
    nivel0Genero = df.genero.unique().tolist()
    
    nivel1Genero = "*"
    
    arvGenero = [nivel0Genero, nivel1Genero]
    
    return arvGenero

In [8]:
def buscaNivelGenero(genero, arvGenero):
    for nivel in range(0, len(arvGenero)):
        if(genero in arvGenero[nivel]):
            return nivel

# Distância entre dois registros

In [9]:
def dist(reg1, reg2):
    genero1 = np.array(reg1[2])
    local1 = np.array(reg1[1].split(", "))
    data1 = np.array(reg1[3].split("/"))
    genero2 = np.array(reg2[2])
    local2 = np.array(reg2[1].split(", "))
    data2 = np.array(reg2[3].split("/"))
    distValor = 0
    if(genero1 != genero2):
        distValor = distValor + 1
    if(data1[2] != data2[2]):
        distValor = distValor + 3
    else:
        if(data1[1] != data2[1]):
            distValor = distValor + 2
        else:
            if(data1[0] != data2[0]):
                distValor = distValor + 1
    if(local1[2] != local2[2]):
        distValor = distValor + 3
    else:
        if(local1[1] != local2[1]):
            distValor = distValor + 2
        else:
            if(local1[0] != local2[0]):
                distValor = distValor + 1
    return (distValor, reg2[0])

# Clusterização dos grupos

In [10]:
def agrupa(df, k):
    grupos = []
    agrupado = []
    dfModificado = np.random.permutation(df)
    for registro1 in range(0, dfModificado.shape[0]-1):
        if(dfModificado[registro1, 0] not in agrupado):
            distancia = []
            for registro2 in range(registro1+1, dfModificado.shape[0]):
                if (dfModificado[registro2, 0] not in agrupado):
                    distancia.append(dist(dfModificado[registro1], dfModificado[registro2]))
            dtype = [('distancia', int), ('id', int)]
            values = distancia
            distancia = np.array(values, dtype = dtype)
            distancia = np.sort(distancia, order='distancia')
            if(len(distancia)+1>=k):
                cluster = [dfModificado[registro1,0]]
                agrupado.append(dfModificado[registro1,0])
                for semelhante in range(0, k-1):
                    cluster.append(distancia[semelhante][1])
                    agrupado.append(distancia[semelhante][1])
                grupos.append(cluster)
            else:
                restantes = [dfModificado[registro1]]
                for restante in range(0, len(distancia)):
                    restantes.append(df[distancia[restante][1]-1])
                    agrupado.append(df[distancia[restante][1]-1][0])
                grupoAdicionado = []
                for reg in restantes:
                    clusterProximo = []
                    if(len(grupos) == len(grupoAdicionado)):
                        grupoAdicionado = []
                    for i in range(0, len(grupos)):
                        if(grupos[i][0] not in grupoAdicionado):
                            clusterProximo.append(dist(reg, df[grupos[i][0]-1]))
                    valuesCluster = clusterProximo
                    clusterProximo = np.array(valuesCluster, dtype = dtype)
                    clusterProximo = np.sort(clusterProximo, order='distancia')
                    for i in range(0, len(grupos)):
                        if(clusterProximo[0][1]==grupos[i][0]):
                            grupos[i].append(reg[0])
                            agrupado.append(reg[0])
                            grupoAdicionado.append(grupos[i][0])
    return grupos

# Generalização do Grupo

In [11]:
def anonimizarGrupo(df, grupo):
    grupoAnonimo = []
    for registro in grupo:
        grupoAnonimo.append(df[registro-1])
    grupoAnonimo = np.array(grupoAnonimo)
    for registro in grupoAnonimo:
        registro[1] = np.array(registro[1].split(', '))
        registro[3] = np.array(registro[3].split('/'))
    variacaoPais = False
    variacaoEstado = False
    variacaoCidade = False
    variacaoAno = False
    variacaoMes = False
    variacaoDia = False
    variacaoGenero = False
    for i in range (1, grupoAnonimo.shape[0]):
        if(grupoAnonimo[i-1][1][2]!=grupoAnonimo[i][1][2]):
            variacaoPais = True
        if(grupoAnonimo[i-1][2]!=grupoAnonimo[i][2]):
            variacaoGenero = True
        if(grupoAnonimo[i-1][3][2]!=grupoAnonimo[i][3][2]):
            variacaoAno = True
    if(not variacaoPais):
        for i in range (1, grupoAnonimo.shape[0]):
            if(grupoAnonimo[i-1][1][1]!=grupoAnonimo[i][1][1]):
                variacaoEstado = True
    if(not variacaoEstado):
        for i in range (1, grupoAnonimo.shape[0]):
            if(grupoAnonimo[i-1][1][0]!=grupoAnonimo[i][1][0]):
                variacaoCidade = True
    if(not variacaoAno):
        for i in range (1, grupoAnonimo.shape[0]):
            if(grupoAnonimo[i-1][3][1]!=grupoAnonimo[i][3][1]):
                variacaoMes = True
    if(not variacaoMes):
        for i in range (1, grupoAnonimo.shape[0]):
            if(grupoAnonimo[i-1][3][0]!=grupoAnonimo[i][3][0]):
                variacaoDia = True

    if(variacaoAno):
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][3] = "**/**/****"
    elif(variacaoMes):
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][3] = "**/**/"+grupoAnonimo[i][3][2]
    elif(variacaoDia):
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][3] = "**/"+grupoAnonimo[i][3][1]+"/"+grupoAnonimo[i][3][2]
    else:
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][3] = grupoAnonimo[i][3][0]+"/"+grupoAnonimo[i][3][1]+"/"+grupoAnonimo[i][3][2]

    if(variacaoPais):
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][1] = "*, *, *"
    elif(variacaoEstado):
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][1] = "*, *, "+grupoAnonimo[i][1][2]
    elif(variacaoCidade):
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][1] = "*, "+grupoAnonimo[i][1][1]+", "+grupoAnonimo[i][1][2]
    else:
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][1] = grupoAnonimo[i][1][0]+", "+grupoAnonimo[i][1][1]+", "+grupoAnonimo[i][1][2]

    if(variacaoGenero):
        for i in range (0, grupoAnonimo.shape[0]):
            grupoAnonimo[i][2] = "*"

    return grupoAnonimo

# Gerar Dataset

In [12]:
def datasetCompleto(df, k):
    df = np.array(df)
    
    grupos = agrupa(df, k)
    
    dataset = []
    for grupo in grupos:
        grupo = anonimizarGrupo(df, grupo)
        for registroAnonimo in grupo:
            dataset.append(registroAnonimo)

    dataset = pd.DataFrame(dataset, columns=['id','local','genero','data','salario'])

    dataset = dataset.drop('id', axis=1)
    
    arquivo = 'salario_'+str(k)+'.csv'
    
    dataset.to_csv(arquivo, index=False)
    
    dataset = np.array(dataset)
    
    return dataset

# Função de Precisão

In [13]:
def precisao(df, N, D, arvores): #N: número de atributos anonimizados = 3, D: número de registros, h: lista com altura da árvore de cada atributo, HGV: lista com alturas máximas das hierarquias dos atributos
    somatorio = 0
    for i in range(0,N):
        for j in range(0,D):
            if(i == 0):
                h = buscaNivelLocal(df[j][i], arvores[0])
                HGV = len(arvores[0])
            elif(i == 1):
                h = buscaNivelGenero(df[j][i], arvores[1])
                HGV = len(arvores[1])
            else:
                h = buscaNivelData(df[j][i], arvores[2])
                HGV = len(arvores[2])
            somatorio += h/HGV
    resultado = 1 - (somatorio)/(D*N)
    return resultado

# Execução

In [14]:
salario = pd.read_csv('salario.csv')

In [15]:
df = preProcessamento(salario)

In [16]:
arvores = gerarArvoreLocal(df), gerarArvoreGenero(df), gerarArvoreData(df)

In [17]:
dataset = datasetCompleto(df, 2)

In [18]:
print(precisao(dataset, 3, dataset.shape[0], arvores))

0.6876666666666666
